In [3]:
def multiply_matrices(A, B):
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
        raise ValueError("Matrix dimensions are not compatible for multiplication.")

    result = [[0 for _ in range(cols_B)] for _ in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]
    return result

def transpose_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    transposed = [[0 for _ in range(rows)] for _ in range(cols)]
    for i in range(rows):
        for j in range(cols):
            transposed[j][i] = matrix[i][j]
    return transposed

def inverse_2x2_matrix(matrix):
    if len(matrix) != 2 or len(matrix[0]) != 2:
        raise ValueError("This inverse function can only handle 2x2 matrices.")

    a, b = matrix[0][0], matrix[0][1]
    c, d = matrix[1][0], matrix[1][1]

    determinant = a * d - b * c

    if determinant == 0:
        raise ValueError("Matrix is singular (determinant is zero) and cannot be inverted.")

    inv_det = 1.0 / determinant
    return [[d * inv_det, -b * inv_det],
            [-c * inv_det, a * inv_det]]

def fit_linear_regression(X_list, y_list):
    # Ensure y is a list of lists (column vector)
    y_matrix = [[val] for val in y_list]

    # Add a bias (intercept) term to X: prepend a column of ones
    X_b = [[1] + row for row in X_list]

    # Calculate X_b.T
    X_b_T = transpose_matrix(X_b)

    # Calculate (X_b.T @ X_b)
    XTX = multiply_matrices(X_b_T, X_b)

    # *** IMPORTANT LIMITATION EXPLANATION ***
    # For a pure Python implementation of matrix inverse, this demo handles only 2x2 matrices
    # (i.e., one feature + intercept). Implementing general N x N matrix inversion from scratch
    # is very complex and computationally intensive (e.g., using Gaussian elimination or adjoint method).
    # Libraries like NumPy are highly optimized for this purpose and are recommended for practical applications.
    if len(XTX) == 2 and len(XTX[0]) == 2:
        try:
            XTX_inv = inverse_2x2_matrix(XTX)
        except ValueError as e:
            print(f"Error: {e}. Cannot compute inverse. Check for multicollinearity or linearly dependent features.")
            return None
    else:
        print("\n--- LIMITATION NOTICE ---")
        print("Error: This pure Python matrix inversion demo is limited to 2x2 matrices only (one feature + intercept).")
        print("For models with more features, a general N x N matrix inverse implementation is required, which is significantly more complex.")
        print("Please adjust your test data to have only one feature, or consider using a library like NumPy for general matrix inversion.")
        print("-------------------------\n")
        return None

    # Calculate (X_b.T @ y)
    XTy = multiply_matrices(X_b_T, y_matrix)

    # Calculate coefficients: theta = (XTX_inv @ XTy)
    theta_matrix = multiply_matrices(XTX_inv, XTy)

    # Return coefficients as a flat list
    return [val[0] for val in theta_matrix]

def predict_linear_regression(X_list, coefficients):
    if coefficients is None:
        raise Exception("Coefficients are not available. Model has not been fitted or fit failed.")

    # Add a bias term to the new data for prediction
    X_b_pred = [[1] + row for row in X_list]

    # coefficients is a flat list, convert to list of lists (column vector) for multiplication
    coeffs_matrix = [[c] for c in coefficients]

    # Predict y values: y_pred = X_b * coefficients
    y_pred_matrix = multiply_matrices(X_b_pred, coeffs_matrix)

    # Return predictions as a flat list
    return [val[0] for val in y_pred_matrix]

print("Linear regression functions defined (pure Python, limited 2x2 matrix inverse).")


Linear regression functions defined (pure Python, limited 2x2 matrix inverse).


In [7]:
# Take input for X (features)
X_input_str = input("Enter comma-separated values for X (e.g., 1,2,3,4,5): ")
try:
    # Convert input string to a list of lists for X, handling a single feature
    X_user = [[float(val.strip())] for val in X_input_str.split(',') if val.strip()]
except ValueError:
    print("Invalid input for X. Please enter numeric values separated by commas.")
    X_user = []

# Take input for y (target)
y_input_str = input("Enter comma-separated values for y (e.g., 2,4,5,4,5): ")
try:
    # Convert input string to a list for y
    y_user = [float(val.strip()) for val in y_input_str.split(',') if val.strip()]
except ValueError:
    print("Invalid input for y. Please enter numeric values separated by commas.")
    y_user = []

if X_user and y_user and len(X_user) == len(y_user):
    print("\nUser-provided X:", X_user)
    print("User-provided y:", y_user)

    print("\nFitting linear regression model with user data...")
    coefficients_user = fit_linear_regression(X_user, y_user)

    if coefficients_user is not None:
        print("\nModel Coefficients (from user data):")
        print(f"Intercept (b0): {coefficients_user[0]:.4f}")
        print(f"Coefficient for X1 (b1): {coefficients_user[1]:.4f}")

        # Prompt for new data to predict
        X_new_input_str = input("\nEnter new comma-separated values for X to predict (e.g., 6,7,8): ")
        try:
            X_new_user = [[float(val.strip())] for val in X_new_input_str.split(',') if val.strip()]
        except ValueError:
            print("Invalid input for new X. Please enter numeric values separated by commas.")
            X_new_user = []

        if X_new_user:
            print("\nMaking predictions for new user data:")
            predictions_user = predict_linear_regression(X_new_user, coefficients_user)

            print("\nPredictions:")
            for i, pred_val in enumerate(predictions_user):
                # Assuming X_new_user[i][0] for single feature
                print(f"For X = {X_new_user[i][0]}, Predicted y = {pred_val:.4f}")
        else:
            print("No valid new X values provided for prediction.")
    else:
        print("Model fitting failed for user data. Check input for consistency and ensure it's suitable for 2x2 inverse.")
else:
    print("Cannot fit model: X and y must be provided, contain valid numbers, and have the same number of data points.")


Enter comma-separated values for X (e.g., 1,2,3,4,5): 1,2,3,4,5
Enter comma-separated values for y (e.g., 2,4,5,4,5): 2,3,4,5,6

User-provided X: [[1.0], [2.0], [3.0], [4.0], [5.0]]
User-provided y: [2.0, 3.0, 4.0, 5.0, 6.0]

Fitting linear regression model with user data...

Model Coefficients (from user data):
Intercept (b0): 1.0000
Coefficient for X1 (b1): 1.0000

Enter new comma-separated values for X to predict (e.g., 6,7,8): 4.5

Making predictions for new user data:

Predictions:
For X = 4.5, Predicted y = 5.5000
